<a href="https://colab.research.google.com/github/manastewary2022/railway_trackprediction_test/blob/ReactMain/RailwayTrackPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import base64
import requests
import csv

# Function to convert image to base64
def image_to_base64(file_path):
    with open(file_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

# List of image file paths to test
image_files = [
    r'/content/drive/MyDrive/RailWayTrackImages/Defective/IMG_20201114_100159.jpg',
    r'/content/drive/MyDrive/RailWayTrackImages/Defective/IMG_20201114_100209.jpg',
    r'/content/drive/MyDrive/RailWayTrackImages/Non defective/IMG_20201114_101633.jpg'
]

# Convert each image to base64 and store them in a list
base64_images = [image_to_base64(image_file) for image_file in image_files]

# Send a POST request with Base64 images in the request body as JSON
url = 'https://api-route-hackathon2024-innovativethought-ai.rhh24cluster-dal10-b3c-32-b7fa127339e34d1cc8d1b3e2be243d89-0000.us-south.containers.appdomain.cloud/multipredict'
data = {'images': base64_images}

try:
    response = requests.post(url, json=data)
    response.raise_for_status()  # Check for HTTP errors
    # Print the response
    print(response.json())

    # Save response to CSV
    def save_response_to_csv(response_data, csv_file_path):
        with open(csv_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            # Assuming response_data is a list of dictionaries
            if isinstance(response_data, list) and len(response_data) > 0:
                # Write headers
                writer.writerow(response_data[0].keys())
                # Write data
                for entry in response_data:
                    writer.writerow(entry.values())

    # Example usage
    csv_file_path = 'prediction_results.csv'
    save_response_to_csv(response.json(), csv_file_path)

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


[{'image_index': 0, 'predicted_class': 'fault', 'probability': 0.00027770339511334896}, {'image_index': 1, 'predicted_class': 'fault', 'probability': 0.000279062835033983}, {'image_index': 2, 'predicted_class': 'fault', 'probability': 0.00029044129769317806}]


In [ ]:
pip install dash pandas plotly


In [ ]:
import dash
from dash import dcc, html
import dash.dependencies as dd
import pandas as pd
import plotly.express as px

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/RailWayTrackImages/prediction_results.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Fault Prediction Dashboard", style={'textAlign': 'center'}),

    dcc.Dropdown(
        id='severity-filter',
        options=[{'label': severity, 'value': severity} for severity in df['severity'].unique()],
        value='High',  # Default value
        clearable=False,
        style={'width': '50%', 'margin': '0 auto'}
    ),

    dcc.Graph(id='fault-severity-pie-chart'),

    dcc.Graph(id='fault-prediction-bar-chart')
])

# Callback for updating the pie chart based on the severity filter
@app.callback(
    dd.Output('fault-severity-pie-chart', 'figure'),
    [dd.Input('severity-filter', 'value')]
)
def update_pie_chart(severity):
    filtered_df = df[df['severity'] == severity]
    pie_chart = px.pie(filtered_df, names='prediction', title=f"Fault Predictions with Severity: {severity}")
    return pie_chart

# Callback for the bar chart to display fault prediction percentage by zone
@app.callback(
    dd.Output('fault-prediction-bar-chart', 'figure'),
    [dd.Input('severity-filter', 'value')]
)
def update_bar_chart(severity):
    filtered_df = df[df['severity'] == severity]
    bar_chart = px.bar(
        filtered_df,
        x='zone',
        y='fault_prediction_percentage',
        color='prediction',
        title=f"Fault Prediction Percentage by Zone for Severity: {severity}",
        labels={'fault_prediction_percentage': 'Fault Prediction %', 'zone': 'Zone'}
    )
    return bar_chart

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>